## Ajuste na tabela

In [0]:
from pyspark.sql import functions as F
# Lê a bronze unificada
dfb = spark.table("sinesp.bronze.municipios_raw_row")

# Deriva ano/mes a partir de mes_ano quando year/month estão inválidos
mes_ano_date = F.to_date(F.col("mes_ano"))   # ex.: 2018-01-01
ano_fix = F.when(F.col("year").cast("int") > 0, F.col("year").cast("int")) \
           .otherwise(F.year(mes_ano_date))
mes_fix = F.when(F.col("month").cast("int").between(1,12), F.col("month").cast("int")) \
           .otherwise(F.month(mes_ano_date))

municipios_silver = (
    dfb
    .withColumn("ano", ano_fix)
    .withColumn("mes", mes_fix)
    .select(
        F.col("cod_ibge").cast("long").alias("cod_ibge"),
        F.initcap("municipio").alias("municipio"),
        F.upper("uf").alias("uf_sigla"),
        F.upper("regiao").alias("regiao"),
        "ano", "mes",                             
        F.col("vitimas").cast("int").alias("vitimas")
    )
    .where(F.col("ano").isNotNull() & F.col("mes").between(1,12))
)

display(municipios_silver)


## Salvar na Silver

In [0]:
(municipios_silver.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema","true")
    .saveAsTable("sinesp.silver.municipios_vitimas"))

print("✓ sinesp.silver.municipios_vitimas criada/atualizada (ano/mes corrigidos)")